In [1]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer

/users/daychman/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
file = '/tempo/merged_clean.csv'

#convert df into a numpy array
df = pd.read_csv(file)

# convert the relationships into categories
rel_cat = df['Relation'].astype('category')

# label each unique value in the column 'Relation' with a number
labels = rel_cat.cat.codes.to_numpy()
df['Relation'] = labels
conversion_dict = np.array(list(enumerate(rel_cat.cat.categories)))

unique, count = np.unique(labels, return_counts=True)
faulty_labels = unique[count < 50]

# get the indices of falty labels
faulty_indices = [i for i in range(len(labels)) if labels[i] in faulty_labels]
df.drop(faulty_indices, inplace=True)


In [5]:
df.to_csv('/tempo/df.csv', index=False)

In [23]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [51]:
sentences = df['Sentence'].to_numpy()

In [52]:
embeddings = model.encode(sentences, show_progress_bar=True)

Batches: 100%|██████████| 45008/45008 [2:24:10<00:00,  5.20it/s]  
